### Елизавета Клыкова, БКЛ181
# Домашнее задание № 3. Определение топика

1. Определить широкий топик текста с помощью gensim. На семинаре мы познакомились с "обычным" встроенным дженсимом, но у него есть улучшенная версия - mallet, для запуска которой надо скачать zip file, распаковать его, дать обертке для маллета path к файлу и запустить. (1 балл - базовая версия, 2 балла - версия с маллетом).
2. Создать функцию или серию функций для подбора оптимального числа групп (1 балл за нахождение оптимального числа, еще 1 балл - если это реализовано через функцию).
3. Создать функцию, которая будет определять для каждого текста только один широкий топик, самый главный. Для этого нужно создать счетчик, и каждый раз, когда в тексте будет встречаться одно из слов, соответсвующих данной теме, добавляйте к счетчику его вес. (2 балла)
4. Получив несколько групп (наборов текста с общим топиком), посчитать внутри каждой группы tf_idf для каждого текста. Для этого нужно взять все тексты с одной темой за корпус и посчитать для каждого из них tf_idf. (3 балла)
5. Для каждого текста определить слова с 5-ю самыми высокими tf_idf и записать их в таблицу.
6. Вывод - таблица excel, csv или pandas, содержащая текст, его широкий топик и 5 tf_idf слов. (1 балл)
7. Логичность и красота кода. (1 балл)
8. Бонус: описать работу coherence score на русском (просто словами).

## Пункты 1-2: широкий топик и оптимальное число

Перед началом работы импортируем все необходимые модули.

In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [2]:
import re
import numpy as np
import pandas as pd
import spacy
import os
import math
import operator

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.wrappers import LdaMallet

# Sklearn
from sklearn.feature_extraction.text import TfidfVectorizer

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim

from collections import OrderedDict
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings('ignore')

In [3]:
# NLTK
import nltk
from nltk import corpus
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

Считаем файл с данными и преобразуем его в pandas dataframe.

*Здесь опять все падает при переносе строки, извините.*

In [4]:
link = 'https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json'
df = pd.read_json(link)
target_names = df.target_names.unique()

1:80: E501 line too long (82 > 79 characters)


C помощью регулярных выражений очистим тексты от ненужных токенов.

In [5]:
data = df.content.values.tolist()

# Remove Emails
data = [re.sub('[\S]*@[\S]*[\s]?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('[\s]+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

4:18: W605 invalid escape sequence '\S'
4:24: W605 invalid escape sequence '\S'
4:29: W605 invalid escape sequence '\s'
7:18: W605 invalid escape sequence '\s'


Токенизируем тексты:

In [6]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

In [7]:
data_words = list(sent_to_words(data))

Создадим модели для выделения би- и триграмм:

In [8]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

Избавимся от стоп-слов, создадим биграммы и лемматизируем тексты:

In [9]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc))
             if word not in stop_words] for doc in texts]

In [10]:
data_words_nostops = remove_stopwords(data_words)

In [11]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

In [12]:
data_words_bigrams = make_bigrams(data_words_nostops)

In [13]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc
                          if token.pos_ in allowed_postags])
    return texts_out

In [14]:
# Оставляем только часть, отвечающую за pos-tagging
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [15]:
# Лемматизируем, оставляя только сущ., прил., глаголы и наречия
data_lemmatized = lemmatization(
    data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

Записываем лемматизированные тексты в dataframe:

In [16]:
df['lem_text'] = data_lemmatized

In [17]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

Пытаемся запустить mallet, все падает, тратим 3 часа на решение проблемы, переходим в Colab, там та же проблема, пишем преподавателям, они разрешают не использовать mallet и обещают не снимать за это балл..................

In [21]:
mallet_path = r'C:\\mallet-2.0.8\\bin\\mallet'  # update this path
ldamallet = gensim.models.wrappers.LdaMallet(
    mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

CalledProcessError: Command 'C:\\mallet-2.0.8\\bin\\mallet import-file --preserve-case --keep-sequence --remove-stopwords --token-regex "\S+" --input C:\Users\7045~1\AppData\Local\Temp\bcc68b_corpus.txt --output C:\Users\7045~1\AppData\Local\Temp\bcc68b_corpus.mallet' returned non-zero exit status 1.

В общем, я пыталась, но придется вернуться с обычному gensim'у.

Чтобы два раза не вставать, начнем с подбора оптимального числа топиков, а затем на этом числе обучим финальный вариант модели. Я увеличиваю passes до 15 и прогоняю модель на кол-ве топиков от 7 до 28 с шагом 3.

In [25]:
def find_opt_amount():
    coherence_dict = {}
    for i in tqdm(range(7, 31, 3)):
        lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                                    id2word=id2word,
                                                    num_topics=i,
                                                    random_state=100,
                                                    update_every=1,
                                                    chunksize=100,
                                                    passes=15,
                                                    alpha='auto',
                                                    per_word_topics=True)
        doc_lda = lda_model[corpus]
        coherence_model_lda = CoherenceModel(model=lda_model,
                                             texts=data_lemmatized,
                                             dictionary=id2word,
                                             coherence='c_v')
        coherence_lda = coherence_model_lda.get_coherence()
        coherence_dict[i] = coherence_lda
    opt_num = max(coherence_dict.items(), key=operator.itemgetter(1))[0]
    return coherence_dict, opt_num

In [26]:
coherences, opt_num = find_opt_amount()

In [27]:
opt_num

7

In [28]:
coherences[opt_num]

0.5318282740836165

Обучим модель на оптимальном числе топиков.

In [29]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=opt_num,
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto',
                                            per_word_topics=True)
doc_lda = lda_model[corpus]

Посмотрим, что получилось:

In [30]:
topics = lda_model.print_topics()
topics

[(0,
  '0.031*"file" + 0.024*"window" + 0.020*"program" + 0.013*"entry" + 0.013*"image" + 0.013*"patient" + 0.011*"problem" + 0.011*"disk" + 0.010*"screen" + 0.010*"run"'),
 (1,
  '0.015*"people" + 0.013*"would" + 0.012*"say" + 0.009*"write" + 0.009*"may" + 0.008*"think" + 0.008*"make" + 0.007*"believe" + 0.007*"reason" + 0.007*"know"'),
 (2,
  '0.010*"kill" + 0.007*"child" + 0.007*"people" + 0.006*"say" + 0.006*"report" + 0.006*"attack" + 0.006*"hockey" + 0.006*"israeli" + 0.006*"fire" + 0.006*"war"'),
 (3,
  '0.025*"line" + 0.020*"write" + 0.016*"would" + 0.014*"be" + 0.014*"go" + 0.013*"get" + 0.012*"article" + 0.010*"good" + 0.010*"know" + 0.010*"nntp_poste"'),
 (4,
  '0.047*"key" + 0.017*"public" + 0.014*"internet" + 0.014*"encryption" + 0.013*"church" + 0.012*"message" + 0.011*"security" + 0.011*"government" + 0.010*"chip" + 0.009*"ripem"'),
 (5,
  '0.692*"ax" + 0.007*"tube" + 0.004*"voltage" + 0.003*"volt" + 0.003*"therapy" + 0.003*"rlk" + 0.002*"lq" + 0.002*"capacitor" + 0.002*

## Пункт 3: один топик для каждого текста
Преобразуем выдачу gensim'а в словарь для удобства дальнейшей работы.

In [31]:
topic_dict = OrderedDict()
for topic in topics:
    topic_dict[topic[0]] = topic[1]

In [32]:
for topic in topic_dict:
    weight_dict = {}
    word_list = topic_dict[topic].split(' + ')
    for w in word_list:
        weight = float(w.split('*')[0])
        word = w.split('*')[1].strip('"')
        weight_dict[word] = weight
    topic_dict[topic] = weight_dict

Мы получили словарь, в котором каждому топику соответствует вложенный словарь слов с их весами. Теперь зададим функцию, определяющую главный топик текста по словам, которые в нем встречаются. Для каждого слова будем проверять, входит ли оно в один из получившихся топиков, и если да, добавлять его вес к общему весу очередного топика в данном тексте. Главным топиком станет тот, вес которого будет больше всего.

In [33]:
def find_one_topic(text, topics):
    topic_counter = {}
    for topic in topics:
        topic_weight = 0
        for word in text:
            if word in topics[topic]:
                topic_weight += topics[topic][word]
        topic_counter[topic] = topic_weight
    main_topic = max(topic_counter.items(), key=operator.itemgetter(1))[0]
    return main_topic, round(topic_counter[main_topic], 4)

Запустим функцию на всех лемматизированных текстах, записывая топики и их веса в два списка.

In [34]:
def one_topic_for_texts(data, topics):
    main_topics = []
    topic_weights = []
    for text in data:
        topic, weight = find_one_topic(text, topics)
        main_topics.append(topic)
        topic_weights.append(weight)
    return main_topics, topic_weights

In [35]:
topics, weights = one_topic_for_texts(data_lemmatized, topic_dict)

Запишем топики и веса в датафрейм. Добавим также столбец index (понадобится для дальнейшей работы).

In [36]:
df['main_topic'] = topics
df['topic_weight'] = weights
df['index'] = range(1, len(df) + 1)
df

,content,target,target_names,lem_text,main_topic,topic_weight,index
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos,"[where, thing, car, nntp_poste, host, park, li...",3,0.045,1
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware,"[si, poll, final, summary, final, call, si, cl...",6,0.028,2
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware,"[question, engineering, computer, network, dis...",3,0.125,3
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics,"[division, line, host, amber, write, write, ar...",3,0.114,4
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space,"[question, organization, smithsonian_astrophys...",3,0.050,5
...,...,...,...,...,...,...,...
11309,From: jim.zisfein@factory.com (Jim Zisfein) \n...,13,sci.med,"[migraine, city, ny_bis, reply, line, cheap, a...",0,0.078,11310
11310,From: ebodin@pearl.tufts.edu\nSubject: Screen ...,4,comp.sys.mac.hardware,"[problem, screen, blank, sometimes, minor, phy...",0,0.031,11311
11311,From: westes@netcom.com (Will Estes)\nSubject:...,3,comp.sys.ibm.pc.hardware,"[este, mount, case, organization, mail, group,...",3,0.035,11312
11312,From: steve@hcrlgw (Steven Collins)\nSubject: ...,1,comp.graphics,"[line, nntp_poste, host, article, write, boy, ...",3,0.107,11313


## Пункты 4-5: tf_idf текстов и 5 слов с наибольшим значением
Сначала запишем топики и соответствующие им тексты в словарь topic_groups для удобства работы. Леммы соединим в тексты, используя в качестве разделителя пробел. Кроме того, сохраним также и полные тексты, чтобы в конце записать получившиеся данные в новый датафрейм без обращения к текущему.

In [37]:
topic_groups = {}
for topic in set(topics):
    token_texts = list(df['lem_text'].loc[df['main_topic'] == topic])
    ftexts = list(df['content'].loc[df['main_topic'] == topic])
    texts = [' '.join(t) for t in token_texts]
    text_to_ftext = {}
    for i, ftext in enumerate(ftexts):
        text_to_ftext[ftext] = texts[i]
    topic_groups[topic] = text_to_ftext

Напишем функцию, которая получает на вход словарь со всеми текстами, разделенными по теме, считает в каждой группе tf-idf и определяет для каждого текста 5 слов с самым высоким tf-idf. Функция возвращает список списков, в которых лежат полный текст, лемматизированный текст, основной топик и 5 слов с наиболее высоким tf-idf.

In [38]:
vectorizer = TfidfVectorizer()

In [39]:
def get_top_words(vector, feature_names, top_n):
    sorted_nzs = np.argsort(vector.data)[:-(top_n+1):-1]
    return feature_names[vector.indices[sorted_nzs]]

In [40]:
all_text_info = []  # здесь будет информация обо всех текстах

for group in tqdm(topic_groups):
    full_texts = list(topic_groups[group].keys())
    texts = [topic_groups[group][f_text] for f_text in full_texts]

    # Находим tf-idf для набора текстов с одним топиком:
    texts_as_vectors = vectorizer.fit_transform(texts)
    feature_names = np.array(vectorizer.get_feature_names())

    # Для каждого текста находим 5 слов с самым высоким tf-idf:
    for i, text in enumerate(texts):
        text_info = []
        article_vector = texts_as_vectors[i, :]
        words = get_top_words(article_vector, feature_names, 5)
        text_info.append(full_texts[i])
        text_info.append(text)
        text_info.append(group)
        text_info.append(words)
        all_text_info.append(text_info)

Полученные данные запишем в новый датафрейм и выведем.

In [41]:
column_names = ['full_text', 'lem_text', 'topic', 'spec_words']
new_df = pd.DataFrame(all_text_info, columns=column_names)
new_df

,full_text,lem_text,topic,spec_words
0,From: kerr@ux1.cso.uiuc.edu (Stan Kerr)\nSubje...,sigma_design double article write look informa...,0,"[board, sigma_design, licensing, have, double]"
1,From: ab@nova.cc.purdue.edu (Allen B)\nSubject...,significance purdue_university line article wr...,0,"[tiff, complexity, image, inability, application]"
2,From: abarden@tybse1.uucp (Ann Marie Barden)\n...,file line question exact entry parameter termi...,0,"[ncd, boot, terminal, control, access]"
3,From: csyphers@uafhp..uark.edu (Chris Syphers)...,size line nntp_poste host uafhp uark alavi wri...,0,"[font, small, uafhp, funky, uark]"
4,From: ab245@cleveland.Freenet.Edu (Sam Latonia...,need phone number western_digital problem orga...,0,"[western_digital, love, crash, phone, copy]"
...,...,...,...,...
11309,From: ph12hucg@sbusol.rz.uni-sb.de (Carsten Gr...,gramme list saarlande rechenzentrum line nntp_...,6,"[drive, slave, master, single, lead]"
11310,From: rcasteto@watsol.uwaterloo.ca (Ron Castel...,line send ticket order information follow team...,6,"[opponent, tor, reply, analyst, discount]"
11311,From: glang@slee01.srl.ford.com (Gordon Lang)\...,help identify video company research laborator...,6,"[template, signal, rgb, exist, video]"
11312,From: CCMB <CCMB@MUSICA.MCGILL.CA>\nSubject: W...,ccmb system use line nntp_poste host small pro...,6,"[ccmb, utility, pro, game, system]"


## Бонус: описание работы coherence score
Topic coherence отражает уровень семантической близости слов, которые имеют наибольший вес внутри одного топика. Для каждого топика эта величина вычисляется как среднее либо медиана коэффициента близости всех пар N наиболее частотных слов. Если семантическая близость велика, то такой топик понятен человеку, а значит, его выделение обосновано. Если же она мала, то такой топик мог быть выделен просто потому, что при обучении модели ее попросили выделить больше топиков, чем нужно для того корпуса, который был дан на вход. Coherence score - среднее topic coherence всех топиков. Чем выше coherence score, тем качественнее модель.